In [1]:
from bs4 import BeautifulSoup
from collections import Counter # Счетчик частот

import re # Регулярные выражения.
import pymorphy2 # Морфологический анализатор.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
#df_train = pd.read_csv('data/train.csv', nrows = 10000, sep='\t', encoding='utf-8')
df_train = pd.read_csv('data/train.csv', sep='\t', encoding='utf-8')

In [3]:
#df_words_name = pd.read_csv('words/words_name_new.txt', nrows = 5, encoding='utf-8')
#df_words_desc = pd.read_csv('words/words_description.txt', nrows = 5, encoding='utf-8')

df_words_name = pd.read_csv('words/words_name_new.txt', encoding='utf-8')
df_words_desc = pd.read_csv('words/words_description.txt', encoding='utf-8')

In [4]:
df_words_name.head(3)

,name
0,менеджер
1,продажа
2,продавец-консультант


In [5]:
df_words_desc_col = df_words_desc.copy()
df_words_desc_col['col_description'] = df_words_desc_col['description'].map(lambda stt: 'desc_' + stt)

df_words_name_col = df_words_name.copy()
df_words_name_col['col_name'] = df_words_name_col['name'].map(lambda stt: 'col_' + stt)
#df_words_name_col['name'].values.tolist()
#df_words_name['name'][2]
df_words_desc_col['col_description'][3]

'desc_активный'

In [6]:
range(len(df_words_name_col))

range(0, 136)

In [7]:
#Тестовая строка для обрабоки
str_t = '<p><strong>Обязанности:</strong></p> <ul> <li><strong>Обеспечение бесперебойной работы сетей водоснабжения и водоотведения, отопления и вентиляции.</strong></li> <li><strong>Проведение текущих и капитальных ремонтов работ оборудования, зданий и т.д.</strong></li> </ul> <p><strong>Требования:</strong></p> <ul> <li>Ответственность</li> <li>Высшее образование</li> </ul> <strong>Условия:</strong> <ul> <li>пятидневная рабочая неделя. Соц.пакет</li> </ul> <p>Звонить в рабочее время с пн-пт</p>'
lll = [p.text for p in BeautifulSoup(str_t, "html5lib").find_all("li")]
lll_text = ' '.join(lll)
#BeautifulSoup(str_t, "html5lib").get_text()
lll

['Обеспечение бесперебойной работы сетей водоснабжения и водоотведения, отопления и вентиляции.',
 'Проведение текущих и капитальных ремонтов работ оборудования, зданий и т.д.',
 'Ответственность',
 'Высшее образование',
 'пятидневная рабочая неделя. Соц.пакет']

In [8]:
morph = pymorphy2.MorphAnalyzer() # Создает объект морфоанализатора и загружет словарь.

In [9]:
#posConv = {'ADJF':'_ADJ','NOUN':'_NOUN'}
#meaningfullPoSes = ['ADJF', 'NOUN']

#text_inp = ' '.join(lll)
#word_to_find = 'работа'

#text_inp_ls = [a[0] for a in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text_inp)]
#reswords = []
#for w in text_inp_ls:
#    wordform = morph.parse(w)[0]
#    if wordform.tag.POS in meaningfullPoSes:
#        reswords.append(wordform.normal_form)
#res_c = Counter(reswords)
#res_c[word_to_find]

In [10]:
posConv = {'ADJF':'_ADJ','NOUN':'_NOUN'}
meaningfullPoSes = ['ADJF', 'NOUN']

def counterSet(text_inp):
    text_inp_ls = [a[0] for a in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text_inp)]
    reswords = []
    for w in text_inp_ls:
        wordform = morph.parse(w)[0]
        if wordform.tag.POS in meaningfullPoSes:
            reswords.append(wordform.normal_form)
    res_c = Counter(reswords)
    #cnt = res_c[word_to_find]
    
    return res_c

In [11]:
counterSet(lll_text)

Counter({'обеспечение': 1,
         'бесперебойный': 1,
         'работа': 2,
         'сеть': 1,
         'водоснабжение': 1,
         'водоотведение': 1,
         'отопление': 1,
         'вентиляция': 1,
         'проведение': 1,
         'текущий': 1,
         'капитальный': 1,
         'ремонт': 1,
         'оборудование': 1,
         'здание': 1,
         'том': 1,
         'ответственность': 1,
         'высокий': 1,
         'образование': 1,
         'пятидневный': 1,
         'рабочий': 1,
         'неделя': 1,
         'пакет': 1})

In [12]:
def prepare_ds(inp_df):
    out_df = inp_df.copy()
    #out_df['descr_li'] = out_df['description'].map(lambda stt: [p.text for p in BeautifulSoup(stt, "html5lib").find_all("li")])
    out_df['description'] = out_df['description'].map(lambda stt: BeautifulSoup(stt, "html5lib").get_text())
    #out_df['n1_менеджер'] = out_df['name'].map(lambda stt: findWordOne(stt, 'менеджер'))
    out_df['n1_counterSet_name'] = out_df['name'].map(lambda stt: counterSet(stt))
    out_df['n1_counterSet_desc'] = out_df['description'].map(lambda stt: counterSet(stt))
        
    for i in range(len(df_words_name_col)):
        out_df[df_words_name_col['col_name'][i]] = \
        out_df['n1_counterSet_name'].map(lambda cts: cts[df_words_name_col['name'][i]])
    
    for d in range(len(df_words_desc_col)):
        out_df[df_words_desc_col['col_description'][d]] = \
        out_df['n1_counterSet_desc'].map(lambda cts: cts[df_words_desc_col['description'][d]])
    
    out_df = out_df.drop(['description', 'name', 'n1_counterSet_name', 'n1_counterSet_desc'], axis=1)
        
    return out_df

In [13]:
df_train_work = prepare_ds(df_train)

In [14]:
#df_train_work.to_csv('df_train_work.csv', index=False)

In [15]:
df_train_work.head(10).T

,0,1,2,3,4,5,6,7,8,9
id,0,1,2,3,4,5,6,7,8,9
target,1,0,0,0,1,0,1,1,1,0
col_менеджер,0,0,0,0,0,0,0,0,1,1
col_продажа,0,0,0,0,0,0,0,0,1,0
col_продавец-консультант,0,0,0,0,1,0,0,0,0,0
col_торговый,0,0,0,0,0,0,1,1,0,0
col_представитель,0,0,0,0,0,0,1,1,0,0
col_клиент,0,0,0,0,0,0,0,0,0,1
col_магазин,1,0,0,0,0,0,0,0,0,0
col_работа,0,0,0,0,0,0,0,0,0,1


In [16]:
X_train, y_train = df_train_work.drop(['target','id'], axis=1).values, df_train_work.target.values

In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import roc_auc_score, roc_curve

In [18]:
model_DT = DecisionTreeClassifier(criterion='entropy', random_state=123)
#model_KN = KNeighborsClassifier()
#model_LR = LogisticRegression(random_state=123)

model_DT.fit(X_train,y_train)
#model_KN.fit(X_train,y_train)
#model_LR.fit(X_train,y_train)

DT_y_hat = model_DT.predict(X_train)
DT_y_hat_proba = model_DT.predict_proba(X_train, )
#KN_y_hat = model_KN.predict(X_train)
#KN_y_hat_proba = model_KN.predict_proba(X_train, )
#LR_y_hat = model_LR.predict(X_train)
#LR_y_hat_proba = model_LR.predict_proba(X_train, )

In [19]:
print(roc_auc_score(y_train, DT_y_hat_proba[:, 1]))
#print(roc_auc_score(y_train, KN_y_hat_proba[:, 1]))
#print(roc_auc_score(y_train, LR_y_hat_proba[:, 1]))

0.9998692354458083


In [20]:
#df_test = pd.read_csv('data/test.csv', nrows = 10000, sep='\t', encoding='utf-8')
df_test = pd.read_csv('data/test.csv', sep='\t', encoding='utf-8')

In [21]:
df_test_work = prepare_ds(df_test)

In [22]:
#df_test_work.to_csv('df_test_work.csv', index=False)

In [23]:
from sklearn.cross_validation import cross_val_score

scores = cross_val_score(model_DT, X_train, y_train, cv = 10)
itog_val = pd.DataFrame([['DecisionTreeClassifier',scores.mean()]], columns=['model', 'scores'])

itog_val.head()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


,model,scores
0,DecisionTreeClassifier,0.943785


In [26]:
X_test_f = df_test_work.drop(['id'], axis=1).values
Y_test_DT_f = model_DT.predict(X_test_f)
Y_test_DT_proba_f = model_DT.predict_proba(X_test_f)

In [25]:
result_DT = df_test_work.loc[:,('id',)]
#берем единички
result_DT['target'] = list(map(lambda x: x[1], Y_test_DT_proba_f))
result_DT.to_csv('result_hw04.csv', index=False)

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold, KFold
from sklearn.metrics import classification_report, confusion_matrix

cv = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

'''
param_grid = [
  {
   'n_estimators': [200, 500, 1000],
   'max_features': ['auto', 'sqrt', 'log2'],
   'max_depth': np.linspace(10,20,11,dtype='int'), 
   'criterion' :['gini', 'entropy'],
   'class_weight': [None],
  },
 ]
'''

param_grid = [
  {
   'n_estimators': np.linspace(180,220,5,dtype='int'), 
   'max_features': ['auto'],
   'max_depth': np.linspace(11,14,4,dtype='int'),  
   'criterion' :['entropy'],
   'class_weight': [None],
   # 'min_samples_split': [3, 5, 10], 
  },
 ]


model_RFC = RandomForestClassifier(random_state=42)
model_RFC.fit(X_train,y_train)

#grid_search_RFC = GridSearchCV(model_RFC, param_grid=param_grid, cv=cv, scoring='roc_auc', n_jobs=-1, verbose=10)

#grid_search_RFC.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [36]:
cross_val_score(model_RFC, X_train, y_train, cv = 10).mean()

0.9609799996721625

In [37]:
X_test_f = df_test_work.drop(['id'], axis=1).values
Y_test_RFC_f = model_RFC.predict(X_test_f)
Y_test_RFC_proba_f = model_RFC.predict_proba(X_test_f)

result_RFC = df_test_work.loc[:,('id',)]
#берем единички
result_RFC['target'] = list(map(lambda x: x[1], Y_test_RFC_proba_f))
result_RFC.to_csv('result_hw04_RFC.csv', index = False)